# SESAR db to JSON dump

In [1]:
import psycopg2
import requests
import json

In [2]:
def get_connection():
    try:
        return psycopg2.connect(
            database="sesardb20240525",
            user="postgres",
            password="smrpostgis",
            host="127.0.0.1",
            port=5432,
        )
    except:
        return False


In [3]:
def executeQuery(conn, querystring):
# CREATE A CURSOR USING THE CONNECTION OBJECT
    curr = conn.cursor()
    # EXECUTE THE SQL QUERY
    curr.execute(querystring)
    # FETCH ALL THE ROWS FROM THE CURSOR
    data = curr.fetchall()
    return data

In [4]:
def getFields(tableName):
#  get field names for table
    fieldsquery="SELECT column_name,ordinal_position "\
     "FROM information_schema.columns " + \
     "WHERE table_schema = 'public' AND table_name = '" + \
        tableName + "' ORDER BY ordinal_position"
    print(fieldsquery)
    fields = executeQuery(conn, fieldsquery)

    fieldlist=[]
    for row in fields:
        fieldlist.append(row[0])
    return fieldlist

In [5]:
def getTableData(tableName, tablePK):
    fl=getFields(tableName)
    #print(tableName, " fields ", fl)
    selectRecordQuery = "SELECT * FROM public." + tableName 
    #print("record query: ", selectRecordQuery)

    data = executeQuery(conn, selectRecordQuery)
    # do something with THE RECORDS 
    print (tableName, " number of records: ", len(data))

    theDict={}
    rows=[]
    rowNum = 0
    for row in data:
        theobj={}
        arecord={}
        for nc in range(len(row)):
            if row[nc] is None:
                # don't put null values in JSON
                continue
            theobj[fl[nc]]=row[nc]
        if tablePK=='noPK':
            arecord['id'+str(rowNum)]=theobj
        else:
            arecord[theobj[tablePK]]=theobj
        #print(arecord)
        rows.append(arecord)
        rowNum = rowNum+1
    
    #print(rows)
    theDict[tableName] = rows
    return theDict

In [6]:
def getPrimaryKey(tableName):
    pkquery = "SELECT c.column_name " + \
        " FROM information_schema.key_column_usage AS c " + \
        " LEFT JOIN information_schema.table_constraints AS t " + \
        " ON t.constraint_name = c.constraint_name " + \
        " WHERE t.table_name = '" + tableName + "' AND t.constraint_type = 'PRIMARY KEY'; " 
    #print(pkquery)
    key = executeQuery(conn, pkquery)
    print("the primary key: ", key)
    if len(key) == 0:
        return "noPK"
    else:
        return key[0][0]

In [7]:
# database connection is global variable.
conn = get_connection()
if conn:
    print("Connection to the PostgreSQL established successfully.")
else:
    print("Connection to the PostgreSQL encountered and error.")

Connection to the PostgreSQL established successfully.


# set up to process a table

data = executeQuery(conn, selClassification)
    # do something with THE RECORDS 
print (len(data))

theDict={}
rows=[]
for row in data:
    theobj={}
    arecord={}
    for nc in range(len(row)):
        theobj[fl[nc]]=row[nc]
    arecord[theobj[tablePK]]=theobj
    #print(arecord)
    rows.append(arecord)

#print(rows)
theDict[tableName] = rows
#print(theDict)



In [8]:
# list of tables in database SELECT * FROM information_schema.tables WHERE table_schema = 'public' and table_type = 'BASE TABLE'
#  get field names for table
tablesquery="SELECT table_name " + \
 "FROM information_schema.tables " + \
 "WHERE table_schema = 'public' AND table_type = 'BASE TABLE'" 
print(tablesquery)
tables = executeQuery(conn, tablesquery)

tablelist=[]
for row in tables:
    tablelist.append(row[0])
print(tablelist)


SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' AND table_type = 'BASE TABLE'
['annual_stats', 'archive_lkup', 'archive_mapping', 'classification', 'download_history', 'cruise_field_prgrm_lkup', 'collection_method_lkup', 'collector_lkup', 'gfz_registered_namespace', 'country_id_map', 'gfz_igsns_need_fix', 'groups', 'ecl_orcid_temp', 'country', 'batch_history', 'launch_type', 'peng_org', 'sesar_user', 'platform_name_lkup', 'nav_type', 'platform_type_lkup', 'primary_location_name_lkup', 'primary_location_type_lkup', 'registrar_lkup', 'sample_publication_url', 'sample_delete_request', 'sample_external_identifier', 'sample_type', 'sample_registration_stats', 'sample_upload_history', 'sesar_role', 'sesar_user_code', 'sample_additional_name', 'sample_doc', 'temp_backfill_publish_date', 'temp_pp', 'v_sql_statement', 'group_sample', 'sesar_user_code_role', 'spatial_ref_sys', 'sample']


In [9]:
for tableName in tablelist:
    print("table: ", tableName)
    
    theKey=getPrimaryKey(tableName)  
    thisTable = getTableData(tableName, theKey)
    
    fileName='SESAR_JSON/'+tableName + '.json'
    with open(fileName, 'w') as outfile:
        json.dump(thisTable, outfile,indent=2, default=str)
    outfile.close()

table:  annual_stats
the primary key:  [('year',)]
SELECT column_name,ordinal_position FROM information_schema.columns WHERE table_schema = 'public' AND table_name = 'annual_stats' ORDER BY ordinal_position
annual_stats  number of records:  20
table:  archive_lkup
the primary key:  []
SELECT column_name,ordinal_position FROM information_schema.columns WHERE table_schema = 'public' AND table_name = 'archive_lkup' ORDER BY ordinal_position
archive_lkup  number of records:  514
table:  archive_mapping
the primary key:  [('archive_mapping_id',)]
SELECT column_name,ordinal_position FROM information_schema.columns WHERE table_schema = 'public' AND table_name = 'archive_mapping' ORDER BY ordinal_position
archive_mapping  number of records:  872
table:  classification
the primary key:  [('classification_id',)]
SELECT column_name,ordinal_position FROM information_schema.columns WHERE table_schema = 'public' AND table_name = 'classification' ORDER BY ordinal_position
classification  number of re

In [10]:
    # CLOSE THE CONNECTION
conn.close()